<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("IMF_MFS.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,freq,indicator,ref_area,indexed_at,provider_code,series_code,series_name,series_num
0,1994,1994-01-01,209.600000,annual,MFS,Monetary and Financial Statistics (MFS),A,FIGB_PA,AM,2025-06-10T03:31:15.139Z,IMF,A.AM.FIGB_PA,"Annual – Armenia – Financial, Interest Rates, ...",Master
1,1995,1995-01-01,114.228889,annual,MFS,Monetary and Financial Statistics (MFS),A,FIGB_PA,AM,2025-06-10T03:31:15.139Z,IMF,A.AM.FIGB_PA,"Annual – Armenia – Financial, Interest Rates, ...",Master
2,2000,2000-01-01,25.511738,annual,MFS,Monetary and Financial Statistics (MFS),A,FIGB_PA,AM,2025-06-10T03:31:15.139Z,IMF,A.AM.FIGB_PA,"Annual – Armenia – Financial, Interest Rates, ...",Master
3,2001,2001-01-01,23.150333,annual,MFS,Monetary and Financial Statistics (MFS),A,FIGB_PA,AM,2025-06-10T03:31:15.139Z,IMF,A.AM.FIGB_PA,"Annual – Armenia – Financial, Interest Rates, ...",Master
4,2002,2002-01-01,17.441590,annual,MFS,Monetary and Financial Statistics (MFS),A,FIGB_PA,AM,2025-06-10T03:31:15.139Z,IMF,A.AM.FIGB_PA,"Annual – Armenia – Financial, Interest Rates, ...",Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [2]:
python_df = pd.read_pickle("IMF_MFS.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,FREQ,REF_AREA,INDICATOR,Frequency,Reference Area,Indicator
0,annual,IMF,MFS,Monetary and Financial Statistics (MFS),A.5X.14____XDC,"Annual – CEMAC – Monetary, Monetary Authoritie...",2000,2000-01-01,1054429.0,1054429.0,A,5X,14____XDC,Annual,CEMAC,"Monetary, Monetary Authorities, Reserve Money ..."
1,annual,IMF,MFS,Monetary and Financial Statistics (MFS),A.5X.14____XDC,"Annual – CEMAC – Monetary, Monetary Authoritie...",2001,2001-01-01,1113041.0,1113041.0,A,5X,14____XDC,Annual,CEMAC,"Monetary, Monetary Authorities, Reserve Money ..."
2,annual,IMF,MFS,Monetary and Financial Statistics (MFS),A.5X.14____XDC,"Annual – CEMAC – Monetary, Monetary Authoritie...",2002,2002-01-01,1271893.0,1271893.0,A,5X,14____XDC,Annual,CEMAC,"Monetary, Monetary Authorities, Reserve Money ..."
3,annual,IMF,MFS,Monetary and Financial Statistics (MFS),A.5X.14____XDC,"Annual – CEMAC – Monetary, Monetary Authoritie...",2003,2003-01-01,1288554.0,1288554.0,A,5X,14____XDC,Annual,CEMAC,"Monetary, Monetary Authorities, Reserve Money ..."
4,annual,IMF,MFS,Monetary and Financial Statistics (MFS),A.5X.14____XDC,"Annual – CEMAC – Monetary, Monetary Authoritie...",2004,2004-01-01,1516178.0,1516178.0,A,5X,14____XDC,Annual,CEMAC,"Monetary, Monetary Authorities, Reserve Money ..."


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [3]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [4]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')
python_df['value'] = python_df['value'].astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [5]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [6]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True